# Projet TextRank
**Avant toute chose, assurez vous de copier ce notebook et de travailler uniquement sur votre copie personnelle**

Préambule
---------

Ce préambule télécharge le dictionnaire word2vec qui associe des mots (`str`) à des vecteurs (`np.array `de taille 200)
Il télécharge aussi le corpus de travail pour ce TP qui est issu de wikipedia



In [ ]:
#INIT notebook
!pip install gensim

import os
import numpy as np
from gensim.models import KeyedVectors
from urllib.request import urlretrieve

#Download word vectors if needed (this may take time)
if 'frw2v.bin' not in os.listdir():
  urlretrieve('https://s3.us-east-2.amazonaws.com/embeddings.net/embeddings/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin','frw2v.bin')
w2v = KeyedVectors.load_word2vec_format("frw2v.bin",binary=True)

#this list stores the 65 most frequent french words, will be used later to avoid too frequent words in sentence vector processing
very_frequent_words = [list(w2v.vocab.keys())[i] for i in range(65)]

In [ ]:
#Téléchargement du corpus wikipedia
!pip install wikipedia
import wikipedia

wikipedia.set_lang('fr')

writers = ['Corneille',"Racine",'Flaubert','Balzac','Zola','Baudelaire','Rimbaud','Verlaine']
corpus  = [ wikipedia.summary(auteur) for auteur in writers ]


  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=e7749a4ccc836b26bf0d65907ad3c5396bdbdb1c711edd9f954ee56c49b7090f
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


A ce stade, la liste `corpus` contient les textes, le dictionnaire `w2v` associe des mots à des vecteurs de taille 100.

Premier exercice: segmentation en phrases et en mots
---------------

Définir une fonction qui découpe un texte en phrases et qui renvoie une liste de phrases.
Définir une autre fonction qui découpe une phrase en mots. Celle-ci renvoie une liste de mots



In [ ]:
#basic functions, might need to make them more complex but work fine for now
def text_tokenize(texte):
  return texte.split(".")

def sent_tokenize(sentence):
  #sentence cleaning
  sentence = sentence.lower()
  sentence = sentence.replace(',', '')
  sentence = sentence.replace('.', '')
  sentence = sentence.replace('(', '')
  sentence = sentence.replace(')', '')
  sentence = sentence.replace("’","'")

  #make clean tokens
  sentence = sentence.split(" ")
  for index in range(len(sentence)):
    sentence[index] = sentence[index].strip()
    if len(sentence[index]) >2 and sentence[index][1] == "'": #delete l', d', j', s', ...
      sentence[index] = sentence[index][2:]

  return sentence


Second exercice: calculer une représentation vectorielle de la phrase par sac de mots
--------------------

Définir une fonction qui prend en argument une liste de mots et qui renvoie un vecteur. Faire attention à gérer le problèmes des mots inconnus du dictionnaire. Faut-il inclure tous les mots pour obtenir un vecteur de phrase informatif ?


Ici on fais le choix de créer un vecteur principal de taille 200(la taille des vecteurs donnés par w2v.).
Pour chaque mot rencontré, si un vecteur lui est associé dans w2v, on additione ce vecteur au vecteur principal. Enfin on divise chaque valeur du vecteur par 1/(nombre de mots), ce qui nous donne un vecteur 'moyen'.

In [ ]:
def vectorize_sent(token_list, vocab):

  #sentence vector
  vector = np.zeros(200)
  count = 0
  for token in token_list:
    if token in vocab and token not in very_frequent_words:
      #sentence vector update
      vector = vector + vocab[token]
      count += 1

  if count == 0:
    return np.zeros(200)
  else:
    #each dimension of sentence vector has to be divided by number of vectors to obtain its "average" value
    return vector.dot(1/count)

Troisième exercice : Définir une fonction qui calcule les similarités entre les phrases
--------------
Définir une fonction qui, à partir d'une liste de phrases, calcule une matrice stochastique construite à partir des similarités cosinus entre chaque couple de phrases. Faut-il réaliser un lissage comme dans le cas du pagerank traditionnel ? pourquoi ?



Ici il n'y a pas besoin d'appliquer un lissage, car aucun des vecteurs de chaque mot de la phrase ne contient 0. Ainsi, après calcul des similarités avec le cosinus, il n'y a pas de valeurs nulle dans la matrice.
Dans le cas d'une approche par sac de mot traditionelle où on compterais les occurences de chaque mot, il est probable que certains vecteurs aient une similarité de 0, car aucun des mots ne correspondrait. Voilà pourquoi on laisse la possibilité de changer la valeur de delta, afin d'appliquer un lissage comme dans le cas du pagerank traditionnel, si jamais on préfèrais d'aborder le problème avec des vecteurs de fréquence.


In [ ]:
def cos_similarity(a,b):
  #keyedvectors class also provides some cosine similarity methods, here we use the standard formula
  return np.dot(a,b)/( np.linalg.norm(a) * np.linalg.norm(b))

def make_matrix(vector_list,delta=0):
  #matrix containing each cosine similarity between all pairs of sentences
  matrix = np.zeros((len(vector_list), len(vector_list)))
  for i in range(0,len(vector_list)):
    for j in range(0, len(vector_list)):
      matrix[i,j] = cos_similarity(vector_list[i], vector_list[j])

  #make the matrix ergodic. By default delta = 0, so nothing changes because here the matrix is already ergodic
  if delta !=0:
    delta = delta/len(vector_list)
  ergod_matrix = np.full(shape = (len(vector_list),len(vector_list)), fill_value = delta, dtype=float)
  matrix = matrix + ergod_matrix

  #make stochastic each vector
  for j in range(len(matrix)):
    matrix[j] = matrix[j].dot(1/np.sum(matrix[j]))

  return matrix


Quatrième exercice : calcul du pageRank
-------------
Définir une fonction qui calcule le pageRank de la matrice stochastique et qui sélectionne les K phrases les plus pertinentes d'un texte

In [ ]:
def pagerank(G,K):

  #1.page rank processing
  X = np.full(len(G), 1/len(G), dtype=float)
  iter = 0
  while iter <10: #after 12 iterations, all values of X seem to be stabilized, setting to 15 just to be sure
    X = X.dot(G)
    iter +=1

  #2. K best scores from vector
  X = list(X)
  best_list = []
  for _ in range(K):
    index = X.index(max(X))
    best_list.append(index) #find best line's index
    X.pop(index)        #remove best line, to find next best line

  #3. return K best sentences
  return best_list


Cinquième exercice : Synthese et commentaires
-------------
Rassemblez les solutions obtenues dans les exercices précédents pour afficher le résumé extractif de chacun des textes du corpus

In [ ]:
def summary(texte):

  #text formating
  texte = text_tokenize(texte)
  sent_list = []
  for sent in texte:
    sent_list.append(sent_tokenize(sent))

  #vector and matrix computing
  vector_list = []
  for index in range(len(sent_list)):
    vector = vectorize_sent(sent_list[index], w2v)
    if vector.all() == 0: #empty sentences
      pass
    else:
      vector_list.append(vector)

  matrix = make_matrix(vector_list)

  #PageRank processing
  best_list = pagerank(matrix, K=2)

  #print summary
  for elt in best_list:
    print(" ".join(sent_list[elt]).strip())
  print("\n\n")

#generate summary for each author
for index in range(len(corpus)):
  print(writers[index])
  summary(corpus[index])


Corneille
[[0.14442932 0.06669722 0.04960989 0.06237024 0.04874629 0.04063723
  0.06894961 0.06811127 0.03027443 0.06648822 0.05825868 0.05323973
  0.05741015 0.08629866 0.07495097 0.02352809]
 [0.07746661 0.16774987 0.061505   0.06026533 0.04764644 0.05400902
  0.06814184 0.07353876 0.04126983 0.07648637 0.0525046  0.04241945
  0.04825925 0.03160543 0.0663038  0.0308284 ]
 [0.04394978 0.04691287 0.12795103 0.07358917 0.05455233 0.073823
  0.07564228 0.07130556 0.05167609 0.05560125 0.07775022 0.04197788
  0.05997694 0.04411765 0.06398662 0.03718734]
 [0.05177513 0.04307294 0.06895556 0.11989448 0.05755832 0.0683242
  0.07700047 0.07695259 0.0610687  0.04294445 0.0526728  0.04994948
  0.07408467 0.05783054 0.06651363 0.03140205]
 [0.04988173 0.04197817 0.06301222 0.07095192 0.14779347 0.06210741
  0.09794116 0.09130705 0.08017335 0.06304871 0.06463114 0.02087245
  0.03419929 0.0331634  0.05139095 0.02754758]
 [0.03945094 0.04514324 0.08089776 0.07990316 0.05892191 0.14021311
  0.077229

Analysez votre démarche et indiquez quels paramètres ont un impact sur le résumé produit au final, comme par exemple:
*  Segmentation et mots inconnus
*  Quels mots inclure dans la vectorisation de la phrase ?
*  Paramètre delta de lissage
* ... (autres)

A votre avis, comment pourrait-on mesurer la qualité d'un résumé ?

La méthode proposée est-elle convaincante ? A votre avis, comment pourrait-on l'améliorer ?  


**note:** ici Balzac renvoie à la page wikipedia Bazar, l'algorithme fonctionne ici aussi. Ce problème vient du module wikipedia lui même qui renvoie à la mauvaise page.

**réglage des paramètres**:

  * les règles de segmentation sont très simples mais semblent fonctionner
  * les mots inconnus sont simplement ignorés par l'algorithme car aucun vecteur ne lui correspond, la plupart sont des nombres (dates) ou bien des mots rares ("racinien", "épurement"), noms propres ("valincour", "ménélik"), ou
  * Mots à exlure: A l'origine, l'idée était de ne prendre en compte que les mots plus longs que 2 caractères, on évitait ainsi de considérer la plupart des déterminants et pronoms. Cette solution a été choisie car w2v ne donne pas d'information sur la classe grammaticale des mots qu'il vectorise. Toutefois, après discussion avec Mathilde Ducos, elle m'a fait remarquer que les mots de w2v.vocab semblaient être triés par ordre de fréquence. Ainsi, on choisi de ne plus considérer les 65 mots les plus fréquents, la plupart n'apportant pas d'information particulière. Ces mots sont stockés dans very_frequent_words, créée au début du programme. L'utilisation de cette méthode ne change que très peu, car la fonction de base fonctionnait assez bien, mais cette approche semble bien plus stable, voilà pourquoi elle a été préférée.

  * Il n'y a pas eu besoin de définir de paramètre delta de lissage, car:
    * chaque vecteur de phrase est de dimension 200, dont toutes les valeurs sont différentes de 0. Dans le cas d'une phrase ne contenant aucun mot reconnu (phrase vide, en langue étrangère), on considère qu'on ne peut pas la traiter et on l'exclue donc du corpus.
    * le calcul de similarité entre deux vecteurs renvoie une valeur entre 0 et 1 à partir du moment où les deux vecteurs partagent au moins une dimension de valeur non nulle. par conséquent aucun de nos calculs de similarité n'a pu résulter à 0.
    * Ainsi, on déduit que la matrice obtenue est déjà ergodique, et qu'il n'est pas nécessaire d'appliquer une matrice de lissage.
    *   **note sur le paramètre delta**: étant donné que la matrice est déjà ergodique, peu importe la valeur de delta, on obtiendra toujours la même matrice finale après l'avoir rendue stochastique. Aussi, il est possible que l'application d'une matrice de lissage ne soit pas nécessaire, car nous n'avons pas forcément besoin de rendre la matrice ergodique. Les besoins de Google lors de l'implémentation de PageRank les ont poussé à rendre leurs matrices ergodiques, mais dans ce cas précis il ne semble même pas nécessaire de faire de même.

**calcul de la qualité d'un résumé:** Afin de déterminer la qualité d'un résumé de texte issu de l'introduction d'un article de wikipedia on peut définir plusieurs paramètres:
  * la présence de caractéristiques essentielles:
    * sujet (ici le titre de l'article),
    * date de naissance et de mort si le sujet est un être humain
    * profession exercée
    * il est possible de détecter  ces informations par le biais de regex pour les dates, de présence de mots clés ("né.e","mort.e",noms de professions exercées)
  * la longueur du résumé, car un résumé trop long n'est plus forcément un résumé. Un résumé trop court pourra passer sous silence certaines informations importantes. Ici on joue sur la longueur du résumé avec le paramètre K de pagerank()


**La méthode implémentée ici** donne un résultat qui n'est pas mauvais, on extrait bien K phrases qui ont le meilleur score selon l'algorithme PageRank, un algorithme qui a fait ses preuves.
 Toutefois nous n'avons aucun contrôle sur les vecteurs utilisés par w2v, ils sont par ailleurs assez compliqué à interpréter, il suffit de faire quelques tests pour s'en rendre compte:
* la similarité entre les vecteurs de "poète" et "poètes" est de 0.68197
* la similarité entre les vecteurs de "poète" et "écrivain" est de 0.73381

Afin d'améliorer la qualité de nos résumés on pourrait proposer plusieurs solutions:
* Sur wikipedia la première phrase est toujours une phrase qui résume très brièvement les informations principales à savoir sur le sujet en question. On pourrait donc obligatoirement mettre cette ligne en tête de notre résumé.
* Afin d'obtenir les phrases qui centralisent au mieux les informations du texte, on pourrait calculer la similarité de chaque vecteur de phrase avec un vecteur général du texte.
* On pourrait aussi calculer la proximité de chaque phrase avec une phrase référence, ou bien un vecteur patron.

Ces deux dernières approches nous éloignent de l'approche de PageRank, mais peut être qu'elles pourraient améliorer la qualité des résumés produits.





